<div align="right">Student: Antoine Moulin</div>

# SD 202
# Computer Lab 2: DB Normalization and Querying

<b>Lab's authors:</b> Albert Biffet, Marie Al-Ghossein <br/>
<b>Date:</b> September 28, 2018

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3    # Python interface for SQLite databases

In [2]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
Clients2:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
Locations2:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
Producer1:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
Production1:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
Sales2:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
Wine1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Wine2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


From this point we will use __%%sql__ magic

In [4]:
%load_ext sql
%sql sqlite:///wine.db

'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [5]:
%%sql SELECT *
FROM Master1
LIMIT 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [6]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

Master1, which is already in the 1NF (the table is flat), has the following functional dependencies:

* NP $\rightarrow$ NOM, PRENOM, REGION 
* NV $\rightarrow$ CRU, DEGRE, MILL
* (NV, NP) $\rightarrow$ NOM, PRENOM, REGION, CRU, DEGRE, MILL, QTE

Thus, we note that (NV, NP) is a key of the table Master1. Besides, NOM, PRENOM & REGION are dependent on the key part NP, and CRU, DEGRE & MILL are dependent on the key part NV, so Master1 is not in 2F. In order to so, we can split the table Master1 into these three new tables below:


**<center>Producer1</center>**

|*Attribute*| *Description*                  |
| --------- | ------------------------------ |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |


**<center>Wine1</center>**

|*Attribute*| *Description*                  |
| --------- | ------------------------------ |
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |


**<center>Production1</center>**

|*Attribute*| *Description*                  |
| --------- | ------------------------------ |
| NP        | Producer number                |
| NV        | Wine number                    |
| QTE       | Number of bottles harvested    |

Doing so, the attributes of the table Producer1 only depend on the key NP, the attributes of Wine1 on NV and the attribute QTE of Production1 on (NV, NP). Thus, these three tables are in NF2.

Actually, they are also in NF3 because, with Producer1 for instance, a producer could come from the same REGION, have the same NOM and the same PRENOM than another producer and yet be someone different. Then, the attributes only depend of the key NP. We could do the same reasoning for the tables Wine1 and Production1.

<u> Note : </u> In fact, we could say that Wine1 is not in NF3 because the attribute DEGRE depends on (CRU, MILL), which is not a key, but let assume we do not have any knowledge about oenology (eventhough we could check that fact).


#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

This time, Master2 has the following functional dependencies:

* NV $\rightarrow$ CRU, DEGRE, MILL
* NB $\rightarrow$ NOM, PRENOM, TYPE
* LIEU $\rightarrow$ REGION
* NV, NB $\rightarrow$ DATES, LIEU, QTE

Then, we observe that (NV, NB) is a key of the table Master2. We can create four new tables listed below. As for the first exercise, we show that these tables are all in NF3:


**<center>Wine2</center>**

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |


**<center>Clients2</center>**

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |

          
**<center>Locations2</center>**

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| LIEU      | Place where the wine was sold                          |
| REGION    | Administrative Region (different to production region) |


**<center>Sales2</center>**

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| NB        | Client (buveur) number                                 |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [7]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [8]:
print("\nContent of the database")
printSchema(conn)


Content of the database
Clients2:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
Locations2:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
Producer1:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
Production1:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
Sales2:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
Wine1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Wine2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
dummy:
  0: DEGRE(NUM)


In [9]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [10]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [11]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [12]:
%%sql DROP TABLE IF EXISTS Producer1;

-- Create Producer1 table
CREATE TABLE Producer1 AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM Master1
WHERE NP IS NOT NULL
ORDER BY NP;

SELECT *
FROM Producer1;

 * sqlite:///wine.db
Done.
Done.
Done.


NP,NOM,PRENOM,REGION
1,Bohn,Rene,Alsace
2,Boxler,Albert,Alsace
3,Six,Paul,Alsace
4,Stentz,Fernand,Alsace
5,Joudeat,Lucien,Bourgogne
6,Marmagne,Bernard,Bourgogne
7,Lasnier,Pierre,Bourgogne
8,Lioger d'Harduy,Gabriel,Bourgogne
9,Moniot,Gilbert,Bourgogne
10,Fournier,Jean Claude,Bourgogne


In [13]:
%%sql DROP TABLE IF EXISTS Wine1;

-- Create Wine1 table
CREATE TABLE Wine1 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM Master1
WHERE NV IS NOT NULL
ORDER BY NV;

SELECT *
FROM Wine1;

 * sqlite:///wine.db
Done.
Done.
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976
6,Mercurey,11.2,1981
7,Grands Echezeaux,11.7,1968
8,Cotes de Beaune Villages,12.3,1975
9,Chapelle Chambertin,11.9,1973
10,Beaujolais Villages,11.8,1979


In [14]:
%%sql DROP TABLE IF EXISTS Production1;

-- Create Production1 table
CREATE TABLE Production1 AS
SELECT DISTINCT NP, NV, QTE
FROM Master1
WHERE NP IS NOT NULL and NV IS NOT NULL
ORDER BY NP, NV;

SELECT *
FROM Production1;

 * sqlite:///wine.db
Done.
Done.
Done.


NP,NV,QTE
1,1,300
1,3,400
1,54,500
2,45,90
2,49,150
2,50,100
2,54,900
4,34,None
4,78,300
5,2,100


#### Exercise 1.4

Create the new tables from Master2:

In [15]:
%%sql DROP TABLE IF EXISTS Wine2;

-- Create Wine2 table
CREATE TABLE Wine2 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM Master2
WHERE NV IS NOT NULL
ORDER BY NV;

SELECT *
FROM Wine2;

 * sqlite:///wine.db
Done.
Done.
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976
6,Mercurey,11.2,1981
7,Grands Echezeaux,11.7,1968
8,Cotes de Beaune Villages,12.3,1975
9,Chapelle Chambertin,11.9,1973
10,Beaujolais Villages,11.8,1979


In [16]:
%%sql DROP TABLE IF EXISTS Clients2;

-- Create Clients2 table
CREATE TABLE Clients2 AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM Master2
WHERE NB IS NOT NULL
ORDER BY NB;

SELECT *
FROM Clients2;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,NOM,PRENOM,TYPE
1,Aristote,None,petit
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
5,Audiberti,Jacques,petit
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
9,Ajar,Emile,petit
10,Andersen,Yann,gros


In [17]:
%%sql DROP TABLE IF EXISTS Sales2;

-- Create Sales2 table
CREATE TABLE Sales2 AS
SELECT DISTINCT NV, NB, DATES, LIEU, QTE
FROM Master2
WHERE NV IS NOT NULL and NB IS NOT NULL
ORDER BY NV, NB;

SELECT *
FROM Sales2;

 * sqlite:///wine.db
Done.
Done.
Done.


NV,NB,DATES,LIEU,QTE
1,2,1977-11-02,BORDEAUX,33
1,44,2015-10-16,PARIS,1
1,45,1983-12-31,RENNES,1
2,48,1983-12-25,LYON,2
3,7,1978-11-01,NICE,6
3,49,1987-12-24,MERCUREY,5
4,8,1982-11-05,TOULOUSE,12
4,44,2015-10-16,PARIS,6
4,50,1981-06-29,SENS,4
5,2,1984-11-04,MERCUREY,50


In [18]:
%%sql DROP TABLE IF EXISTS Locations2;

-- Create Locations2 table
CREATE TABLE Locations2 AS
SELECT DISTINCT LIEU, REGION
FROM Master2
WHERE LIEU IS NOT NULL
ORDER BY LIEU;

SELECT *
FROM Locations2;

 * sqlite:///wine.db
Done.
Done.
Done.


LIEU,REGION
BORDEAUX,NOUVELLE-AQUITAINE
CAEN,NORMANDIE
CANNES,PROVENCE-ALPES-CÔTE D'AZUR
CHALON SUR SAONE,BOURGOGNE-FRANCHE-COMTÉ
DIJON,BOURGOGNE-FRANCHE-COMTÉ
LE CHESNAY,ÎLE-DE-FRANCE
LILLE,HAUTS-DE-FRANCE
LYON,AUVERGNE-RHÔNE-ALPES
MERCUREY,BOURGOGNE-FRANCHE-COMTÉ
NANCY,GRAND EST


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [19]:
%sql SELECT DISTINCT TYPE FROM Clients2;

 * sqlite:///wine.db
Done.


TYPE
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

Here, we show three methods to answer the question. The first two ones are faster than the last one (see here : https://stackoverflow.com/questions/14644807/performance-of-inner-join-vs-cartesian-product) but I couldn't say which of the first two is the most optimized. However, the third one still seems more readable and the first seems more explicit than the second.

In [20]:
%%sql SELECT DISTINCT CRU, REGION 
FROM (Producer1 
      JOIN (SELECT * 
            FROM (Production1 
                  JOIN Wine1 ON Production1.NV = Wine1.NV)) as Temp ON Producer1.NP = Temp.NP)
WHERE CRU = "Pommard" or CRU = "Brouilly"
ORDER BY CRU;

 * sqlite:///wine.db
Done.


CRU,REGION
Brouilly,Bourgogne
Pommard,Bourgogne
Pommard,Rhone


In [21]:
%%sql SELECT CRU, REGION
FROM Production1
NATURAL JOIN Producer1
NATURAL JOIN Wine1
WHERE CRU = "Pommard" or CRU = "Brouilly"
ORDER BY CRU;

 * sqlite:///wine.db
Done.


CRU,REGION
Brouilly,Bourgogne
Pommard,Bourgogne
Pommard,Rhone


In [22]:
%%sql SELECT CRU, REGION
FROM Producer1, Production1, Wine1
WHERE Producer1.NP = Production1.NP and Production1.NV = Wine1.NV and (CRU = "Pommard" or CRU = "Brouilly")
ORDER BY CRU;

 * sqlite:///wine.db
Done.


CRU,REGION
Brouilly,Bourgogne
Pommard,Bourgogne
Pommard,Rhone


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [23]:
%%sql SELECT DISTINCT REGION 
FROM (Producer1 
      JOIN (SELECT * 
            FROM (Production1 
                  JOIN Wine1 ON Production1.NV = Wine1.NV)) as Temp ON Producer1.NP = Temp.NP)
WHERE CRU = "Pommard"
INTERSECT
SELECT DISTINCT REGION 
FROM (Producer1 
      JOIN (SELECT * 
            FROM (Production1 
                  JOIN Wine1 ON Production1.NV = Wine1.NV)) as Temp ON Producer1.NP = Temp.NP)
WHERE CRU = "Brouilly"

 * sqlite:///wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [24]:
%%sql SELECT DISTINCT CRU, MILL, SUM(QTE)
FROM Wine2
NATURAL JOIN Sales2
GROUP BY CRU, MILL
ORDER BY CRU ASC, MILL DESC;

 * sqlite:///wine.db
Done.


CRU,MILL,SUM(QTE)
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,7
Beaujolais Villages,1979,520
Beaujolais Villages,1978,130
Beaujolais Villages,1976,120
Beaujolais Villages,1975,10
Chapelle Chambertin,1973,30
Chateau Corton Grancey,1980,4
Chenas,1984,1


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [25]:
%%sql SELECT NV, NUMBER_OF_PRODUCERS
FROM (SELECT NV, COUNT(NP) as NUMBER_OF_PRODUCERS 
      FROM Production1 
      GROUP BY NV)
WHERE NUMBER_OF_PRODUCERS > 3;

 * sqlite:///wine.db
Done.


NV,NUMBER_OF_PRODUCERS
45,5
78,5
89,4
98,5


#### Exercise 2.6

Which producers have not produced any wine?

In [26]:
%%sql SELECT NP, NOM, PRENOM
FROM Producer1
WHERE Producer1.NP NOT IN (SELECT NP 
                           FROM Production1);

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

Same as 2.2., here are two methods (I know it is quite useless but it helps me to train).

In [27]:
%%sql SELECT DISTINCT Clients2.NB, NOM, PRENOM
FROM (Clients2
JOIN (SELECT * 
      FROM (Sales2
      JOIN Wine2 ON Sales2.NV = Wine2.NV)) as Temp ON Clients2.NB = Temp.NB)
WHERE Temp.MILL = "1980"
ORDER BY Clients2.NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


In [28]:
%%sql SELECT DISTINCT NB, NOM, PRENOM
FROM Clients2
NATURAL JOIN Sales2
NATURAL JOIN Wine2
WHERE MILL = "1980"
ORDER BY NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [29]:
%%sql SELECT DISTINCT NB, NOM, PRENOM
FROM Clients2
WHERE NB NOT IN (SELECT DISTINCT NB
                 FROM Clients2
                 NATURAL JOIN Sales2
                 NATURAL JOIN Wine2
                 WHERE MILL = "1980"
                 ORDER BY NB)

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
9,Ajar,Emile
10,Andersen,Yann
11,Breton,Andre
12,Bataille,Georges


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [31]:
%%sql SELECT DISTINCT NB, NOM, PRENOM
FROM Clients2
NATURAL JOIN Sales2
NATURAL JOIN Wine2
EXCEPT
SELECT DISTINCT NB, NOM, PRENOM
FROM Clients2
NATURAL JOIN Sales2
NATURAL JOIN Wine2
WHERE MILL != 1980;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


#### Exercise 2.10

List all wines from 1980

In [32]:
%%sql SELECT DISTINCT NV, CRU, MILL, DEGRE
FROM Wine2
WHERE MILL = "1980"
ORDER BY NV;

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5
4,Mercurey,1980,10.9
16,Meursault,1980,12.1
20,Cote de Brouilly,1980,12.1
26,Chateau Corton Grancey,1980,None
28,Volnay,1980,11
43,Fleurie,1980,11.4
74,Arbois,1980,12
78,Etoile,1980,12
79,Seyssel,1980,11


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [33]:
%%sql SELECT DISTINCT NV, CRU, MILL, DEGRE
FROM Wine2
NATURAL JOIN Sales2
WHERE MILL = "1980" and NB = 2
ORDER BY NV;

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [34]:
%%sql SELECT NB, NOM, PRENOM
FROM Clients2
NATURAL JOIN Sales2
NATURAL JOIN Wine2
WHERE MILL = "1980"
GROUP BY NB
HAVING COUNT(NB) in (SELECT COUNT(NV)
                     FROM Wine2
                     WHERE MILL = "1980")

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
